# 서울시 교통사고 시각화

In [145]:
import pandas as pd

In [146]:
traffic_Seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [147]:
traffic_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      4500 non-null   int64 
 1   월       4500 non-null   int64 
 2   자치구명    4500 non-null   object
 3   발생건수    4500 non-null   int64 
 4   사망자수    4500 non-null   int64 
 5   부상자수    4500 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 211.1+ KB


In [148]:
# 2019년 데이터만 추출하기
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도'] == 2019]
traffic_Seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158


In [149]:
# index 정리하기
traffic_Seoul_2019.reset_index(drop=True, inplace=True)
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [150]:
import numpy as np

In [151]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기 -1
traffic_analysis = pd.pivot_table(traffic_Seoul_2019[['자치구명','발생건수','부상자수','사망자수']],
                                  index='자치구명',aggfunc=np.sum)
traffic_analysis.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [152]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기-2
traffic_analysis =  traffic_Seoul_2019.pivot_table(['발생건수','부상자수','사망자수'],index=['자치구명'], aggfunc='sum') 
traffic_analysis.head()


,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [153]:
# 위도 경도  데이터 불러오기
seoul_limit = pd.read_csv("../Data/seoul.csv", encoding='euc-kr')
seoul_limit.columns = ['자치구명','lon','lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [154]:
# traffic_Seoul_2019와 seoul_limit 병합하기
data_result = pd.merge(traffic_analysis, seoul_limit, on= '자치구명')
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [155]:
data_result.tail()

,자치구명,발생건수,부상자수,사망자수,lon,lat
20,용산구,1187,1614,10,126.9904,37.53254
21,은평구,1119,1464,7,126.9292,37.60278
22,종로구,1133,1559,6,126.9793,37.57323
23,중구,1184,1615,5,127.0032,37.55273
24,중랑구,1620,2122,12,127.0928,37.60652


In [156]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [157]:
# folium을 사용하기 위해 int를 float로 변환해야한다.
data_result[['발생건수','부상자수', '사망자수']] = data_result[['발생건수','부상자수', '사망자수']].astype(float)
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


### 자치구 구청을 표시하기

In [158]:
import folium 

In [159]:
map = folium.Map(
    location=[37.5502,126.982],
    zoom_start=11
)

# 위치 정보를 Market로 표시
for name, lat, lng in zip(data_result.자치구명, data_result.lat, data_result.lon): # 위치가 정해져있다.
    folium.Marker([lat, lng], popup=name).add_to(map)
map

In [160]:
# 선생님 .ver

map = folium.Map(
    location=[37.5502,126.982],
    zoom_start=11
)

# 위치 정보를 Market로 표시
for n in data_result.index:
    folium.Marker([data_result['lat'][n], data_result['lon'][n]],
                popup=data_result['자치구명'][n]
    ).add_to(map)
map

In [161]:
# 자치구별 교통사고 발생건수 표시하기
map = folium.Map(
    location=[37.5502,126.982],
    zoom_start=11
)
for n in data_result.index:
    folium.Marker([data_result['lat'][n], data_result['lon'][n]],
                popup=data_result['자치구명'][n]
    ).add_to(map)
    folium.CircleMarker([data_result['lat'][n],
                         data_result['lon'][n]],
                         popup=data_result['발생건수'][n],
                         radius=data_result['발생건수'][n]/100,
                         fill_color='skyblue',
                         fill_opacity=0.3,
    ).add_to(map)
    
map

In [162]:
# 자치구별 교통사고 사망건수 표시하기
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722.0,5182.0,14.0,127.0475,37.51731
1,강동구,1414.0,1910.0,11.0,127.1238,37.53013
2,강북구,1277.0,1706.0,7.0,127.0255,37.63975
3,강서구,1829.0,2491.0,20.0,126.8496,37.55094
4,관악구,1363.0,1755.0,10.0,126.9515,37.47834


In [163]:
data_result['사망자수'].min()
data_result['사망자수'].max()

20.0

In [164]:
map = folium.Map(
    location=[37.5502,126.982],
    zoom_start=11
)
n = data_result['사망자수'].argmax()   # index의 최대값
m = data_result['사망자수'].argmin()   # index의 최소값

folium.CircleMarker([data_result['lat'][n],
                         data_result['lon'][n]],
                         popup=data_result['사망자수'][n],
                         radius=data_result['사망자수'][n]*2,
                         color = 'red',
                         fill_color='red',
                         fill_opacity=0.2,
).add_to(map)

folium.CircleMarker([data_result['lat'][m],
                         data_result['lon'][m]],
                         popup=data_result['사망자수'][m],
                         radius=data_result['사망자수'][m]*2,
                         color = 'red',
                         fill_color='red',
                         fill_opacity=0.2,
).add_to(map)
    
map